In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
np.expand_dims??

In [3]:
# kernel python 3.10

In [4]:
print("GPU Available:", tf.config.list_physical_devices('GPU'))

In [5]:
df = pd.read_csv(os.path.join('jigsaw-toxic-comment-classification-challenge','train.csv', 'train.csv'))

In [6]:
df.head()

In [7]:
df.iloc[3]['comment_text']

In [8]:
df[df.columns[2:]].iloc[3]

# 1. Preprocess

In [9]:
from tensorflow.keras.layers import TextVectorization #tokenization|

In [10]:
TextVectorization??

In [11]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [12]:
df.columns

In [13]:
X

In [14]:
y

In [15]:
MAX_FEATURES = 200000 #number of words in the vocab

In [16]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                              output_sequence_length=1800,
                              output_mode='int')

In [17]:
X

In [18]:
type(X)

In [19]:
X.values #mengubah X pandas series ke numpy

In [20]:
vectorizer.adapt(X.values)

In [21]:
vectorizer('Hello Bang')

In [63]:
vectorizer.get_vocabulary()

In [62]:
vectorizer.get_config()

In [64]:
vectorizer_config = vectorizer.get_config()
vectorizer_vocab = vectorizer.get_vocabulary()

In [65]:
import pickle
with open('vectorizer_config.pkl', 'wb') as f:
    pickle.dump(vectorizer_config, f)
with open('vectorizer_vocab.pkl', 'wb') as f:
    pickle.dump(vectorizer_vocab, f)

In [23]:
# np.save('vocabulary.npy', vocab)

In [24]:
vectorized_text = vectorizer(X.values)
vectorized_text

In [25]:
# MCSHBAP -- map, cache, shuffle, batch, prefetch from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks

In [26]:
dataset.as_numpy_iterator().next()

In [27]:
batch_X, batch_y = dataset.as_numpy_iterator().next()

In [28]:
print('Ukuran Batch X', batch_X.shape)
print('Ukuran Batch y', batch_y.shape)

In [29]:
# len(dataset)*16 jml total_data

In [30]:
train = dataset.take(int(len(dataset)*.7)) # 70% dari len dataset
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [31]:
print('Jumal data train', len(train))
print('Jumal data val', len(val))
print('Jumal data test', len(test))

In [32]:
train_generator = train.as_numpy_iterator()

In [33]:
train_generator.next()

# 2. Create Sequential Model

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [35]:
model = Sequential()

#create embedding layers
model.add(Embedding(MAX_FEATURES +1, 32))

# #Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
# model.add(LSTM(32, activation='tanh'))

#Feature extractor fully
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

#final layer
model.add(Dense(6, activation='sigmoid'))

In [36]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [37]:
model.summary()

disaranin epochs 10

In [38]:
history = model.fit(train, epochs=10, validation_data=val)

In [39]:
history.history

In [40]:
from matplotlib import pyplot as plt

In [41]:
plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
plt.show()

# 3. Make Predictions

In [42]:
input_text = vectorizer('You freaking suck! i will to kill you')

In [43]:
df.columns[2:]

In [44]:
# model.predict(np.array([input_text]))
model.predict(np.expand_dims(input_text,0))

In [45]:
batch = test.as_numpy_iterator().next()

In [46]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [47]:
(model.predict(batch_X) > 0.5).astype(int)

In [48]:
res = model.predict(np.expand_dims(input_text, 0))

# 4. Evaluate Model

In [49]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [50]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [51]:
for batch in test.as_numpy_iterator():
    #unpack batch
    X_true, y_true = batch

    #make prediction
    yhat = model.predict(X_true)

    #flatten the prediction
    y_true = y_true.flatten()
    yhat = yhat.flatten()

    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

In [52]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

# 5. Test and Gradio

In [53]:
import tensorflow as tf

In [54]:
!pip install gradio jinja2

In [55]:
import gradio as gr

In [56]:
model.save('toxic-v1.h5')

In [58]:
model = tf.keras.models.load_model('toxic-v1.h5')

In [59]:
input_str = vectorizer('i hate you')

In [60]:
res = model.predict(np.expand_dims(input_str,0))

In [61]:
res

In [ ]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    result = model.predict(vectorized_comment)

    # print kelas klasifikasinya
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, result[0][idx]>0.5)  # Changed results to result

    return text

In [ ]:
interface = gr.Interface(fn=score_comment,
                         inputs=gr.Textbox(lines=2, placeholder='Comment to score'),
                         outputs='text')

harus matiin antivirus

In [ ]:
interface.launch(share=True)